# Regression - Part 1

# Load the data

In [634]:
import numpy as np

X_train_ini = np.load('Xtrain_Regression_Part1.npy')
Y_train_ini = np.load('Ytrain_Regression_Part1.npy')

X_test = np.load('Xtest_Regression_Part1.npy')

# Define cross-validation method

In [635]:
from sklearn.model_selection import RepeatedKFold

round_digits = 7
#random_state = 1
random_state = None

# define model evaluation method (repeats k-folds n times, with k-folds=n_splits and n=n_repeats)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=random_state)

# Set train and validation samples

In [636]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_train_ini, Y_train_ini, 
                                                  random_state=random_state, 
                                                  shuffle=True, test_size=1/3)


# Train the linear regression model

In [637]:
from sklearn.linear_model import LinearRegression

# Ordinary least squares Linear Regression
lr = LinearRegression()
lr.fit(x_train, y_train)

# Training score
R2 = lr.score(x_train, y_train)
print('R²:', round(R2, round_digits))

R²: 0.9994361


# MSE for training data

In [638]:
from numpy import round
from sklearn.metrics import mean_squared_error, r2_score

y_hat_lr_train = lr.predict(x_train)

OLS_lr_MSE_predicted_train = round(mean_squared_error(y_train, y_hat_lr_train, squared=True), round_digits)
print('MSE:', OLS_lr_MSE_predicted_train)
print('R²:', round(r2_score(y_train, y_hat_lr_train), round_digits))

MSE: 0.0085847
R²: 0.9994361


# Print the coeficients and the intercept

In [639]:
print('Coef.:', lr.coef_)
print('Intercept:', lr.intercept_)

Coef.: [[ 0.0346244  -0.01243518  0.08162367  0.33326577 -0.65326003  1.69899781
   0.05504587  1.79735278  0.02208676 -0.0040372  -1.45850423 -0.70648537
   0.02863855 -0.60465255  0.01090581 -0.36618762 -0.13616055 -1.37292635
  -1.29161461  0.96281263]]
Intercept: [-0.00868824]


# Prediction and performance measures

In [640]:
from sklearn.metrics import mean_squared_error, r2_score

# Prediction on validation data
y_pred = lr.predict(x_val)

# Scores on validation data
#print('RMSE: %.5f' % mean_squared_error(y_val, y_pred, squared=False))
#print('SSE:', round(y_pred.size * mean_squared_error(y_val, y_pred, squared=True), 5))

OLS_lr_MSE_predicted_val = round(mean_squared_error(y_val, y_pred, squared=True), round_digits)

print('MSE:', OLS_lr_MSE_predicted_val)
print('R²:', round(r2_score(y_val, y_pred), round_digits))


MSE: 0.0175628
R²: 0.9988211


# Cross-validation performance measures

In [641]:
from sklearn.model_selection import cross_validate, cross_val_score
from numpy import absolute

# Evaluate the model with cross-validation 
score = cross_val_score(lr, x_train, y_train, cv=cv,
                        scoring='neg_mean_squared_error')

OLS_lr_MSE_cv_train = round(absolute(score.mean()), round_digits)

print('score MSE:', score)
print("MSE:", OLS_lr_MSE_cv_train, '  std:', round(score.std(), round_digits))

score MSE: [-0.03130917 -0.0220965  -0.01726631 -0.02416971 -0.01984054 -0.02262922
 -0.01449908 -0.01553112 -0.01147981 -0.01591376 -0.0136434  -0.01166018
 -0.01374315 -0.02489465 -0.03410001 -0.02007365 -0.01723285 -0.00841482
 -0.01293761 -0.02058394 -0.01907184 -0.02430749 -0.01044349 -0.01789852
 -0.00528341 -0.03476852 -0.05104711 -0.02538341 -0.01013346 -0.0080803 ]
MSE: 0.0192812   std: 0.0093706


# Ridge regression

Linear least squares with l2 regularization.

Minimizes the objective function:

$ \ ||y - Xw||^2_2 + alpha * ||w||^2_2 $

# Train rigde regression model

In [642]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from numpy import arange

ridge = Ridge()

parameters = {'alpha': arange(0, 1, 0.01)}

ridge_lr = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=cv, refit=True, verbose=1)
ridge_lr.fit(x_train, y_train)

print('Ridge Coef.:', ridge_lr.best_estimator_.coef_)
print('Ridge Intercept:', ridge_lr.best_estimator_.intercept_)

ridge_MSE_cv_train = round(absolute(ridge_lr.best_score_), round_digits)

print('Ridge best params:', ridge_lr.best_params_ )
print('Ridge MSE:', ridge_MSE_cv_train)

Fitting 30 folds for each of 100 candidates, totalling 3000 fits
Ridge Coef.: [[ 0.03427928 -0.01355077  0.08039951  0.33297524 -0.65138286  1.69693016
   0.05593936  1.79383638  0.02209371 -0.00425667 -1.45582166 -0.70682687
   0.02899441 -0.60429744  0.0114378  -0.36601241 -0.13535202 -1.36995839
  -1.29010011  0.96225611]]
Ridge Intercept: [-0.00877692]
Ridge best params: {'alpha': 0.08}
Ridge MSE: 0.0206884


best_estimator_:

Estimator that was chosen by the search, i.e. estimator which gave highest score (or smallest loss if specified) on the left out data. Not available if refit=False.

best_score_:

Mean cross-validated score of the best_estimator

best_params_:

Parameter setting that gave the best results on the hold out data.

Importantly, we can configure the hyperparameter search to refit a final model with the entire training dataset using the best hyperparameters found during the search. This can be achieved by setting the “refit” argument to True, then retrieving the model via the “best_estimator_” attribute on the search result.


In [643]:
print(ridge_lr.best_score_)
print(ridge_lr.best_estimator_)
print(ridge_lr.best_params_)

-0.020688425944083204
Ridge(alpha=0.08)
{'alpha': 0.08}


In [644]:
y_hat_ridge = ridge_lr.best_estimator_.predict(x_val)

ridge_MSE_predicted_val = round(mean_squared_error(y_val, y_hat_ridge, squared=True), round_digits) 
print('MSE ridge:', ridge_MSE_predicted_val)

MSE ridge: 0.0175963


# Train Lasso regression model

https://medium.com/pursuitnotes/day-34-regularization-in-machine-learning-3-c37c336b079c

Linear Model trained with L1 prior as regularizer (aka the Lasso).

The optimization objective for Lasso is:

$\ (1 / (2 * n samples)) * ||y - Xw||^2_2 + alpha * ||w||_1 $

In [645]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

lasso = Lasso()
#parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}
parameters = {'alpha': arange(0.0009, 0.002, 0.0001)}

lasso_lr = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = cv)
lasso_lr.fit(x_train, y_train)

print('Lasso Coef.:', lasso_lr.best_estimator_.coef_)
print('Lasso Intercept:', lasso_lr.best_estimator_.intercept_)

print('Lasso best params:', lasso_lr.best_params_ )

lasso_MSE_cv_train = round(absolute(lasso_lr.best_score_), round_digits)
print('Lasso MSE:', lasso_MSE_cv_train)

Lasso Coef.: [ 0.03276565 -0.01200024  0.08038537  0.33103384 -0.65249745  1.6985844
  0.05392691  1.79602991  0.02055651 -0.0032033  -1.45727654 -0.70661485
  0.02780045 -0.60405259  0.01059724 -0.36554638 -0.13476266 -1.37112496
 -1.29034953  0.96286233]
Lasso Intercept: [-0.0086366]
Lasso best params: {'alpha': 0.0009}
Lasso MSE: 0.0178631


In [646]:
y_hat_lasso = lasso_lr.best_estimator_.predict(x_val)

lasso_MSE_predicted_val = round(mean_squared_error(y_val, y_hat_lasso, squared=True), round_digits)
print('MSE lasso:', lasso_MSE_predicted_val)

MSE lasso: 0.0175307


# Print all MSE results and save to CSV file

In [647]:
print('################ OLS Regression ################')
print('OLS_lr_MSE_predicted_train:', OLS_lr_MSE_predicted_train)
print('OLS_lr_MSE_cv_train:', OLS_lr_MSE_cv_train)
print('OLS_lr_MSE_predicted_val:', OLS_lr_MSE_predicted_val)
print('')

print('#################### Ridge ####################')
print('ridge_cv_train:', ridge_MSE_cv_train)
print('ridge_predicted_val:', ridge_MSE_predicted_val)
print('')

print('#################### Lasso ####################')
print('lasso_MSE_cv_train:', lasso_MSE_cv_train)
print('lasso_MSE_predicted_val:', lasso_MSE_predicted_val)


################ OLS Regression ################
OLS_lr_MSE_predicted_train: 0.0085847
OLS_lr_MSE_cv_train: 0.0192812
OLS_lr_MSE_predicted_val: 0.0175628

#################### Ridge ####################
ridge_cv_train: 0.0206884
ridge_predicted_val: 0.0175963

#################### Lasso ####################
lasso_MSE_cv_train: 0.0178631
lasso_MSE_predicted_val: 0.0175307


In [652]:
import csv

f = open('AAut_Project_part1_results.csv', mode='a')


writer = csv.writer(f)
sep1 = ';'

row1 = str(OLS_lr_MSE_cv_train) + sep1 + str(OLS_lr_MSE_predicted_val) + sep1 + \
       str(ridge_cv_train) + sep1 + str(ridge_predicted_val) + sep1 + \
       str(lasso_MSE_cv_train) + sep1 + str(lasso_MSE_predicted_val)

# write a row to the csv file
writer.writerow(row1)

# close the file
f.close()